In [1]:
%matplotlib inline

In [2]:
import sys
sys.path.append('../')

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import cv2
import matplotlib.pyplot as plt
from pprint import pprint

In [ ]:
tf.app.flags.DEFINE_string(
    'dataset_dir', None, 'The directory where the dataset files are stored.')

In [ ]:
tf.app.flags.DEFINE_string(
    'split_name', 'train', 'The name of the train/test split.')

In [ ]:
# simulate tf.app.run()
# and parse the argvs
_ = tf.flags.FLAGS._parse_flags(
    ['--dataset_dir', '/Users/yangli/mnt/data/synthtext',])

In [ ]:
FLAGS = tf.app.flags.FLAGS

if not FLAGS.dataset_dir:
    raise ValueError('You must supply the dataset directory with --dataset_dir')

## create network

In [ ]:
# create network
# input of network is a tensor
# the tensor could be :
# 1. placeholder
# 2. variable
# 3. constant
# 4. feature_layers
# please do not care about which kind it is
# it is just a tensor
# for image data format : NHWC or NCHW

In [ ]:
# for notebook, we want break the code into
# lines to interactive run the code
# so that the input of net work could be visualized.
# so, for now, the meaning of the placeholder
# is shown.

# image = tf.placeholder(tf.float32, [None, None, None, 3])

# acturally, the input image size usually is constant

In [4]:
from nets import vgg

In [5]:
images = tf.placeholder(tf.float32, [None, 224, 224, 3])

In [6]:
with tf.Graph().as_default():
    logits, _ = vgg.vgg_16(images, num_classes=1, is_training=True, scope='vgg_16')

In [7]:
variables_to_restore = slim.get_variables_to_restore(exclude=['vgg_16/fc6', 'vgg_16/fc7', 'vgg_16/fc8'])

In [8]:
for var in variables_to_restore:
    print(var.op.name)

vgg_16/conv1/conv1_1/weights
vgg_16/conv1/conv1_1/biases
vgg_16/conv1/conv1_2/weights
vgg_16/conv1/conv1_2/biases
vgg_16/conv2/conv2_1/weights
vgg_16/conv2/conv2_1/biases
vgg_16/conv2/conv2_2/weights
vgg_16/conv2/conv2_2/biases
vgg_16/conv3/conv3_1/weights
vgg_16/conv3/conv3_1/biases
vgg_16/conv3/conv3_2/weights
vgg_16/conv3/conv3_2/biases
vgg_16/conv3/conv3_3/weights
vgg_16/conv3/conv3_3/biases
vgg_16/conv4/conv4_1/weights
vgg_16/conv4/conv4_1/biases
vgg_16/conv4/conv4_2/weights
vgg_16/conv4/conv4_2/biases
vgg_16/conv4/conv4_3/weights
vgg_16/conv4/conv4_3/biases
vgg_16/conv5/conv5_1/weights
vgg_16/conv5/conv5_1/biases
vgg_16/conv5/conv5_2/weights
vgg_16/conv5/conv5_2/biases
vgg_16/conv5/conv5_3/weights
vgg_16/conv5/conv5_3/biases


## read checkpoint file

In [9]:
restorer = tf.train.Saver(variables_to_restore)

In [10]:
with tf.Session() as sess:
    restorer.restore(sess, '../checkpoints/vgg/vgg_16.ckpt')
    print('model restored')

INFO:tensorflow:Restoring parameters from ../checkpoints/vgg/vgg_16.ckpt
model restored


In [11]:
init_fn = slim.assign_from_checkpoint_fn('../checkpoints/vgg/vgg_16.ckpt', variables_to_restore)